# Estimating Metrics for MeshFleet benchmark

## Semantic, Geometric and distributional metrics

In [ ]:
import os 
from metrics.metrics_eval import process_metrics_by_viewpoint
os.environ["OMP_NUM_THREADS"] = "1"

gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
gen_folder = "example_data/Meshfleet_Eval/Results_000"
cfg_path  = "example_data/Meshfleet_Eval/config_meshfleet.json"

metrics_result = process_metrics_by_viewpoint(
    ground_truth_folder=gt_folder,
    generated_folder=gen_folder,
    device="cpu",
    config_path=cfg_path,
)

import json
print(json.dumps(metrics_result, indent=2))

### Write results into json file

In [ ]:
import os
from metrics.metrics_eval import process_metrics_by_viewpoint, tensor_to_serializable, json_file_to_combined_table
json_output = json.dumps(tensor_to_serializable(metrics_result), indent=4)

# Save the results in a JSON file in the root folder.
output_file = os.path.join(os.path.dirname(gt_folder), "meshfleet_metrics_results.json")
with open(output_file, 'w') as f:
    f.write(json_output)

print(f"Metrics saved to {output_file}")

## Prompt alignment metrics

In [ ]:
from metrics.metrics import Metrics, GeometryMetrics, ImageBasedPromptEvaluator
from metrics.helpers import preprocess_image, process_folder_with_prompt, process_folder_with_metadata_file
import os 
import pandas as pd
from tqdm import tqdm
os.environ["OMP_NUM_THREADS"] = "1"

gen_base_folder = "example_data/Meshfleet_Eval/Results_000"
metadata_file_path="example_data/meshfleet_test.csv"

# Example usage
prompt_metric = ImageBasedPromptEvaluator() 
mean_scores, std_scores = process_folder_with_metadata_file(generated_base_folder=gen_base_folder, metadata_file_path=metadata_file_path,
                                                            prompt_metric=prompt_metric, 
                                                            preprocess_image=preprocess_image)
print("Mean scores:")
print(mean_scores)
print("Standard deviation scores:")
print(std_scores)

## Vehicle based dimension comparison

If you want to compare the dimensions without a detailed geometric analysis you can also use `estimate_dimension_differences` from `FlorenceWheelbaseOD`. This method estimates the dimensions and wheelbase of the (vehicle) objects based on the images normalizes them based on the height of the object and compares them. The method returns a dictionary with the differences in the dimensions. The last value is the non normalized height difference. 

In [ ]:
import os 
from metrics.viewpoint_florence import FlorenceWheelbaseOD
from metrics.helpers import evaluate_vehicle_dimensions

os.environ["OMP_NUM_THREADS"] = "1"

# Example usage
generated_base_folder = "example_data/Meshfleet_Eval/Results_000"
metadata_file_path="example_data/meshfleet_test.csv"
florence_wheelbase_od = FlorenceWheelbaseOD()
average_diff, std_diff = evaluate_vehicle_dimensions(generated_base_folder, metadata_file_path, florence_wheelbase_od)
print(f"Average difference in vehicle dimensions: {average_diff}")
print(f"Standard deviation of differences: {std_diff}")